In [1]:
from ImageBlocks import split_image, blocks_similarity
import pandas as pd
from pathlib import Path
import cv2
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from time import sleep

In [2]:
def print_grid(l):
    for i in l:
        for j in i:
            print(j, end=" ")
        print("\n")
    print("-------------------------")

In [3]:
def image_segmentation(blocks, sim, stride, thresh=0.9):
    if not all([2 <= stride[0] <= blocks.shape[0], 2 <= stride[1] <= blocks.shape[1]]):
        print("Invalid Stride Size!")
        raise ValueError

    block_colors = [["-" for j in range(blocks.shape[1])] for i in range(blocks.shape[0])]
    for i in range(blocks.shape[0] - stride[0] + 1):
        print(f"Row{i}")
        for j in range(blocks.shape[1] - stride[1] + 1):
            point_neighbours = similarity_with_neighbours(i, j, blocks)

            for n in point_neighbours:
                x, y = list(map(int, n.split("_")[-2:]))
                if point_neighbours[n] >= thresh and block_colors[x][y] == "-":
                    block_colors[x][y] = "Y"
                else:
                    block_colors[x][y] = "R"

            print_grid(block_colors)

In [2]:
def graph_neighbours_similarity(sim):
    """
    Creates a graph using networkx. Keeps the centre node at the centre and joins it
    with other blocks present in the dictionary. Edges use the similarity scores as
    their labels. Similarity scores are present in the same dictionary as values.
    :param sim: dict
                Returned from similarity_with_neighbours(), contains the centre block
                and its neighbours along with their similarity score.
    :return: None
    """
    # Storing the centre block
    centre = sim["center"]

    # Removing it from the dictionary
    sim.pop("center", None)

    G = nx.Graph()

    G.add_node(centre)
    G.add_nodes_from(list(sim.keys()))
    G.add_edges_from([(centre, i) for i in sim])

    pos = nx.spring_layout(G)

    plt.figure(figsize=(9, 9))
    nx.draw(G, with_labels=True, node_size=4000)
    nx.draw_networkx_edge_labels(G, pos, edge_labels={(centre, i): sim[i] for i in sim}, font_color="red",
                                 label_pos=0.8)

    plt.axis('off')
    plt.show()

In [3]:
def similarity_with_neighbours(row, col, blocks):
    """
    Finds the neighbours of the (row, col) indexed block in our image blocks.
    Then calculates the similarity between the centre block and its neighbours.
    :param row: int
                Row in which our block is present.
    :param col: int
                Column in which our block is present.
    :param blocks: Numpy array
                A list of blocks in which our centre block is present.
    :return: sim: dict
                A dictionary in which block names are the keys and the similarity
                score with the centre acts as their respective values.
                First element pair in this dictionary is (centre, Block_{row}_{col})
    """
    h, w = blocks.shape[0], blocks.shape[1]
    if not all([0 <= row < h, 0 <= col < w]):
        print("Invalid coordinates!")
        return -1

    centre = blocks[row][col]
    pos_index = [-1, 0, 1]
    neighbours_index = [(row-i, col-j) for i in pos_index for j in pos_index if (i, j) != (0, 0)]
    bool_index = [(i, j) for i, j in neighbours_index if all([0 <= i < h, 0 <= j < w])]
    # sim = {"center": f"Block_{row}_{col}"}
    sim = {}
    for i, j in bool_index:
        sim[f"Block_{i}_{j}"] = blocks_similarity(centre, blocks[i][j])


    return sim

In [4]:
path = Path.cwd()/"bottle.jpg"
img_blocks, img = split_image(str(path), block_size=(200, 200), gray=False, draw_img=True)

In [5]:
for i in range(4):
    for j in range(3):
        print(img_blocks[i][j], end=" ")
    print()

0-0 0-1 0-2 
1-0 1-1 1-2 
2-0 2-1 2-2 
3-0 3-1 3-2 


In [6]:
cv2.imshow("original image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
s = similarity_with_neighbours(2, 2, img_blocks)

TypeError: 'Block' object is not subscriptable

In [63]:
graph_neighbours_similarity(s)

KeyError: 'center'

In [6]:
class Block:
    def __init__(self):
        self.name = "Dummy"
        self.data = []
        self.shape = len(self.data)

In [7]:
a = Block()
b = Block()
c = Block()
d = Block()

In [8]:
blocks = np.array([a, b, c, d])

In [9]:
blocks.shape

(4,)

In [3]:
from Segmentation import random_color

In [4]:
random_color()

(231, 73, 208)